In [1]:
# import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import os
from pathlib import Path

# import torchvision
# import torchvision.transforms as transforms
# from torchvision import datasets
# import torchvision.models as models

# import os
import numpy as np
import pandas as pd
from PIL import Image
import time
# import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import roc_auc_score

import albumentations as A
import albumentations.pytorch

In [ ]:
# setting seed
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# setting device to cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# HYPERPARAMS HERE

# params
lr = 3e-3
momentum = 0.9
weight_decay = 3e-3

# transforms
presize = 256
crop = 256

# batch size
batch_size = 32

# n_epochs
frozen = 1
unfrozen = 1

In [ ]:
# transforms
train_transform = A.Compose([
        A.SmallestMaxSize(presize),
        A.RandomCrop(crop, crop),
        A.Normalize(),
        A.HorizontalFlip(),
        A.Rotate(limit=30),
        A.Cutout(),
        albumentations.pytorch.ToTensorV2()])

test_transform = A.Compose([
        A.SmallestMaxSize(presize),
        A.CenterCrop(crop, crop),
        A.Normalize(),
        albumentations.pytorch.ToTensorV2()])

In [ ]:
# dataset class
class dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return (self.df.shape[0])

    def __getitem__(self, index):
        image = Image.open(self.df.fname[index]).convert('RGB')
        image = np.array(image)

        label = torch.tensor(self.df.label[index]).long()  

        if self.transform:
            augmentations = self.transform(image=image)
            image = augmentations['image']

        return image, label

In [1]:
# setting paths
    ROOT = Path.cwd().parent
    # image data files
    DATA = os.path.join(ROOT, 'data')
    # dataframes for training
    INPUT = os.path.join(ROOT, 'input')
    # our input df
    DF_PATH = os.path.join(INPUT, 'train_data.csv')

IndentationError: unexpected indent (<ipython-input-1-2add2b7f0ac0>, line 2)

In [ ]:
df = pd.read_csv(DF_PATH)

train_df = df[df.kfold!=0].reset_index(drop=True)
valid_df = df[df.kfold==0].reset_index(drop=True)

# create dataset
train_dataset = dataset(train_df, train_transform)
test_dataset = dataset(test_df, test_transform)
# create loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=True, 
                                            num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=True, 
                                            num_workers=num_workers)

In [ ]:
# displayig the data (looks this way because of normalization)
batch_tensor = next(iter(train_loader))[0][:10,...]
grid_img = torchvision.utils.make_grid(batch_tensor, nrow=5)
# grid_img.shape
plt.figure(figsize=(16,6))
plt.imshow(grid_img.permute(1, 2, 0)); # uncomment to show

In [ ]:
# just a check if all is good with the shapes of the loaders
print(f'dataloader test: {next(iter(train_loader))[0].shape}, {next(iter(test_loader))[0].shape}')

In [ ]:
# loss
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
# learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
# main train function
def train_model(n_epochs=1,
                model=model,
                train_loader=train_loader,
                test_loader=test_loader,
                criterion=criterion,
                optimizer=optimizer, 
                lr_scheduler=lr_scheduler):

    print(f'================')
    print(f'started training...')

    for epoch in range(n_epochs):

        model.train()
        t0 = time.time()
        epoch_loss = 0

        for batch, (images, labels) in enumerate(train_loader):

            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, labels)
            epoch_loss += loss

            loss.backward()
            optimizer.step()

        epoch_time = round(time.time() - t0)

        lr_scheduler.step()
        test_loss, test_auc = test_model(model, test_loader)
        
        print(f'epoch: [{epoch+1}/{n_epochs}], train loss: {epoch_loss:.3f}, valid loss: {test_loss}, roc auc: {test_auc}, time: {epoch_time//60:.0f}m {epoch_time%60:.0f}s')

    return model

In [ ]:
# function for validation or testing
def test_model(model=model, test_loader=test_loader):

    model.eval()
    epoch_loss = 0
    roc_auc = 0

    with torch.no_grad():

        for batch, (images, labels) in enumerate(test_loader):

            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            epoch_loss += loss
            roc_auc += roc_auc_score_multiclass(labels, preds)

    avg_epoch_loss = round(float(epoch_loss / len(test_loader)), 3)
    avg_roc_auc = round(float(roc_auc / len(test_loader)), 3)

    return avg_epoch_loss, avg_roc_auc

In [ ]:
# unfreeze all the params for training
def unfreeze(model=model):
    for param in model.parameters():
        param.requires_grad = True
    return model